# Cicada Data

Data analysis reproduction concerning the cicada’s genome.

Eric Mossotti  
Jul 6, 2024

To reproduce DNA Zoo’s summary table on the 17-year cicada.

## Introduction

### Problem

The steps involved in reproducing data can be unclear.

### Purpose

To elaborate on the objective stated at the top of this document, the purpose is to supplement the DNA Zoo’s report with a more easily accessible data analysis (DA) pipeline. To accomplish this, I seek to independently reproduce and supplement their article’s table while supplying all data processing steps with documented code embedded in the report itself. Although there’s nothing wrong with the data or article, it could be taken further. \[@dnazoo\], \[@magicica\]

### Stakeholders

This might be of interest to the original authors of the article. More generally, the spirit of this work could transfer to other domains of data intensive research and analytics.

### Source

All data used within this report was freely available from a public database hosted by DNA Zoo. \[@dnazoo.s\]

## Pipeline

In [ ]:
flowchart TB
    A((1)):::circle --> B((2)):::circle
    B --> C((3)):::circle
    C --> D((4)):::circle
    
    subgraph Extract ["1. Extract"]
        direction LR
        A1["Create local\nanalysis directory"] --> A2["Download from web\nto local directory"]
    end
    subgraph Transform ["2. Transform"]
        direction LR
        B1["Decompress\nfasta.gz file"] --> B2["Output text file\nusing bash script"]
        B2 --> B3["Transform text to\nindexed dataframe"]
    end
    subgraph Load ["3. Load"]
        direction LR
        C1["Format table values\nwith thousands-separator"]
    end
    subgraph Present ["4. Present"]
        direction LR
        D1["Create a Styler object\nfrom the dataframe"] --> D2["Dataframe is accessible\nfrom the Styler object"]
        D2 --> D3["Reproduce DNA Zoo table\nusing Python to R conversion"]
    end
    
    A ~~~ Extract
    B ~~~ Transform
    C ~~~ Load
    D ~~~ Present
    
    Extract -.->|fasta.gz| Transform
    Transform -.->|pd.DataFrame| Load
    Load -.->|string variables| Present

flowchart TB
 A((1)):::circle --> B((2)):::circle
 B --> C((3)):::circle
 C --> D((4)):::circle
 
 subgraph Extract ["1. Extract"]
 direction LR
 A1["Create local\nanalysis directory"] --> A2["Download from web\nto local directory"]
 end
 subgraph Transform ["2. Transform"]
 direction LR
 B1["Decompress\nfasta.gz file"] --> B2["Output text file\nusing bash script"]
 B2 --> B3["Transform text to\nindexed dataframe"]
 end
 subgraph Load ["3. Load"]
 direction LR
 C1["Format table values\nwith thousands-separator"]
 end
 subgraph Present ["4. Present"]
 direction LR
 D1["Create a Styler object\nfrom the dataframe"] --> D2["Dataframe is accessible\nfrom the Styler object"]
 D2 --> D3["Reproduce DNA Zoo table\nusing Python to R conversion"]
 end
 
 A ~~~ Extract
 B ~~~ Transform
 C ~~~ Load
 D ~~~ Present
 
 Extract -.->|fasta.gz| Transform
 Transform -.->|pd.DataFrame| Load
 Load -.->|string variables| Present

## Extract

This would be the data extraction phase of the DA pipeline.

### Create Project Directory

In [ ]:
reticulate::source_python("00_Extract/scripts/directorize.py")

In [ ]:
import os

def directorize(base_path, structure):
    
    for dir_name, subdirs in structure.items():
        dir_path = os.path.join(base_path, dir_name)
        os.makedirs(dir_path, exist_ok = True)
        
        for subdir in subdirs:
            subdir_path = os.path.join(dir_path, subdir)
            os.makedirs(subdir_path, exist_ok = True)

In [ ]:
# Define the directory structure
structure = {
    "00_Extract/": ["data/", "scripts/"],
    "01_Transform/": ["data/", "scripts/"],
    "02_Load/": ["data/", "scripts/"],
    "03_Present/": ["data/", "scripts/"]
}

# Create the analysis folder structure in a preferred base directory
# "" = project's working directory
directorize("", structure)

### Download to Local Machine

In [ ]:
reticulate::source_python(
    "00_Extract/scripts/importer.py")

In [ ]:
# ---- Import data from the web with wget
import os
import sys
import wget

def importer (fileMap):
    # Download from URL to path and notify when complete
    for url, file_path in fileMap.items():
        # Checking file existence
        if not os.path.exists(file_path):
            wget.download(url, file_path)
            print(f"{file_path} written")
        else:
            print(f"{file_path} already exists.")

In [ ]:
# Set the url
url = "https://dnazoo.s3.wasabisys.com/Magicicada_septendecula/magicicada_hifiasm.asm.bp.p_ctg_HiC.fasta.gz"

# Set the local file path
fpath = "00_Extract/data/magicicada.fasta.gz"

# Map the url to the file path
fileMap = {url: fpath}

importer(fileMap)

00_Extract/data/magicicada.fasta.gz already exists.

> **The specific link used to download all data from**
>
> <https://dnazoo.s3.wasabisys.com/Magicicada_septendecula/magicicada_hifiasm.asm.bp.p_ctg_HiC.fasta.gz>

## Transform

The data transformation phase of the pipeline.

### Decompress .GZ

In [ ]:
reticulate::source_python("01_Transform/scripts/decompress.py")

In [ ]:
# ---- Decompress the gz file with gzip

import os
import gzip
import shutil

def decompress(gzFasta, fasta):
    
    # If not decompressed, then decompress and redirect to a new file path
    if not os.path.exists(fasta):
        # File doesn't exist, then decompress
        with gzip.open(gzFasta, 'rb') as f_in:
            with open(fasta, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"{fasta} has been decompressed and written.")
    else:
        print(f"The file {fasta} already exists. Skipping unzip.")

In [ ]:
# Set the compressed fasta.gz file variable
gzFasta = "00_Extract/data/magicicada.fasta.gz"

# Set the decompressed fasta file variable
fasta = "01_Transform/data/magicicada.fasta"

# Pass file paths to the function
decompress(gzFasta, fasta)

The file 01_Transform/data/magicicada.fasta already exists. Skipping unzip.

This chunk should be ran locally instead of with `quarto render`. When working with the source file, change the code-chunk language specifier from `{.bash}` back to `{bash}`. You might have to add the `{bash}` tag entirely to the div. Not sure how else to go about accomplishing this within my current Quarto project setup. \[@trizna2020\]

``` bash
# BASH SCRIPT

# The uncompressed fasta file variable
fasta=01_Transform/data/magicicada.fasta

# The text file path variable generated by the script
summary_stats=01_Transform/data/summary_stats.txt

assembly_stats $fasta > $summary_stats
```

Transform the text file into a Python dataframe. I am opting to not to blanket change data-types as output format could vary by user preference.

In [ ]:
# Import  external python script to local library environment
reticulate::source_python("01_Transform/scripts/assemblyFramer.py")

In [ ]:
""" Utilizes Python string methods and multi-indexing 
to process assembly_stats' output text file """

import pandas as pd
import re

def assemblyFramer(statsPath = None):
    
    #---- Read Text File
    with open(statsPath, 'r') as file:
        content = file.read()
        
    #---- Regex Matching
    pairs = re.findall(r"\"\w+\"\:\s\d*\.?\d*", content)
    
    #---- Clean Strings
    cleaned_list = [pair.replace('"', '').replace(':', '').strip() for pair in pairs]
    
    #---- Split Strings
    labeled_list = [item.split() for item in cleaned_list]
    
    #---- Create DataFrame
    df = pd.DataFrame(labeled_list, columns = ['Label', 'Value'])
    
    #---- Add Category Column
    df['Category'] = ['Contigs'] * 17 + ['Scaffolds'] * 17
    
    #---- Create Arrays
    category_array = pd.Series.to_list(df['Category'])
    label_array = pd.Series.to_list(df['Label'])
    value_array = pd.Series.to_list(df['Value'])
    
    #---- Combine Arrays to List
    arrayList = [category_array, label_array]
    
    #---- Define Multi-Level Indices
    indices = pd.MultiIndex.from_arrays(arrays = arrayList, names = ('Category', 'Label'))
    
    #---- Index a DataFrame 
    df_indexed = pd.DataFrame(data = value_array, index = indices)
    
    #---- Rename Non-Indexed Column
    df_indexed = df_indexed.rename(columns = {0:"Value"})
    
    return df_indexed

In [ ]:
# Set the local text file path
statsPath = "01_Transform/data/summary_stats.txt"
# Run to yield an multi-indexed dataframe
df = assemblyFramer(statsPath)

## Load

This is the data loading phase. Following completion of this stage, querying the data should be more intuitive than before.

In [ ]:
reticulate::source_python("02_Load/scripts/strint.py")

In [ ]:
""" Formats the string representation of an 
integer value as a comma separated string. """

import pandas as pd
import re


# Format output with comma seperator for thousands place
def strint (dataframe, category, label):
    
    # Find the desired value
    stat = dataframe.loc[(category, label), "Value"]
    
    # Set boolean match value
    isFloat = re.search(r"\.", stat)
    
    # Convert to float if there is a decimal
    if isFloat:
        stat = pd.to_numeric(stat, downcast = "float")
    else:
        # Else convert to an integer 
        stat = pd.to_numeric(stat, downcast = "integer")
        
    # Add a thousands seperator and convert back to a string
    stat = f'{stat:,}'
    
    return stat 

In [ ]:
#---- Contigs
ctig_len = strint(df, "Contigs", "total_bps")
ctig_count = strint(df, "Contigs", "sequence_count")
ctig_n50 = strint(df, "Contigs", "N50")
ctig_max = strint(df, "Contigs", "longest")

#---- Scaffolds
sfld_len = strint(df, "Scaffolds", "total_bps")
sfld_count = strint(df, "Scaffolds", "sequence_count")
sfld_n50 = strint(df, "Scaffolds", "N50")
sfld_max = strint(df, "Scaffolds", "longest")

> **Python query syntax made easier**
>
> ``` python
> # strint(dataframe, category, label)
> ctig_len = strint(df, "Contigs", "N50")
>
> # --->> 
>
> # Which then finds the desired value or 'Value'
> stat = dataframe.loc[(category, label), "Value"]
> ```

## Present

### The Pandas Table

This is a slightly formatted view of the Pandas table designed to be more easily queried to return the desired statistic. If, however, you’d like to treat the Styler object as the unchanged, dataframe object, use the `forma_df.data` syntax.

[:The original dataframe output:](#NutFrame)

``` r
reticulate::source_python("03_Present/scripts/formaFrame.py")
```

``` python
#---- Count Colors
# Create color palettes for first level index and second level index + columns.
n_colors1 = len(df.index.levels[0])
n_colors2 = len(df.index.levels[1]) + len(df.columns)

#---- Palettes
# Adjust color palettes easily
palette1 = sns.color_palette("Pastel2", n_colors = n_colors1)
palette2 = sns.color_palette("husl", n_colors = n_colors2)

#---- Palette-Index Dictionaries
# Map index levels and column names to colors with dictionaries
color_dict1 = dict(zip(df.index.levels[0], palette1))
# For index level 2 and value columns, using a different palette
color_dict2 = dict(zip(list(
    df.index.levels[1]) + list(df.columns), palette2))

#---- Call Function
forma_df = formaFrame(df, color_dict1, color_dict2)

#---- Hide Headers
# The columns headers look a bit odd for display purposes
forma_df = forma_df.hide(axis = "index", names = True)
forma_df = forma_df.hide(axis = "columns", level = 0)

#---- Display 
forma_df
```

<style type="text/css">
#T_440a4_row0_col0, #T_440a4_row17_col0 {
  background-color: rgba(246, 112, 136, 0.2);
}
#T_440a4_row1_col0, #T_440a4_row18_col0 {
  background-color: rgba(247, 117, 67, 0.2);
}
#T_440a4_row2_col0, #T_440a4_row19_col0 {
  background-color: rgba(213, 140, 49, 0.2);
}
#T_440a4_row3_col0, #T_440a4_row20_col0 {
  background-color: rgba(187, 151, 49, 0.2);
}
#T_440a4_row4_col0, #T_440a4_row21_col0 {
  background-color: rgba(163, 159, 49, 0.2);
}
#T_440a4_row5_col0, #T_440a4_row22_col0 {
  background-color: rgba(135, 167, 49, 0.2);
}
#T_440a4_row6_col0, #T_440a4_row23_col0 {
  background-color: rgba(79, 176, 49, 0.2);
}
#T_440a4_row7_col0, #T_440a4_row24_col0 {
  background-color: rgba(50, 176, 114, 0.2);
}
#T_440a4_row8_col0, #T_440a4_row25_col0 {
  background-color: rgba(52, 174, 144, 0.2);
}
#T_440a4_row9_col0, #T_440a4_row26_col0 {
  background-color: rgba(53, 172, 164, 0.2);
}
#T_440a4_row10_col0, #T_440a4_row27_col0 {
  background-color: rgba(54, 170, 181, 0.2);
}
#T_440a4_row11_col0, #T_440a4_row28_col0 {
  background-color: rgba(56, 168, 201, 0.2);
}
#T_440a4_row12_col0, #T_440a4_row29_col0 {
  background-color: rgba(59, 163, 236, 0.2);
}
#T_440a4_row13_col0, #T_440a4_row30_col0 {
  background-color: rgba(137, 148, 244, 0.2);
}
#T_440a4_row14_col0, #T_440a4_row31_col0 {
  background-color: rgba(186, 130, 244, 0.2);
}
#T_440a4_row15_col0, #T_440a4_row32_col0 {
  background-color: rgba(231, 102, 244, 0.2);
}
#T_440a4_row16_col0, #T_440a4_row33_col0 {
  background-color: rgba(245, 99, 211, 0.2);
}
#T_440a4_level0_row0 {
  background-color: rgba(179, 226, 205, 0.2);
}
#T_440a4_level1_row0, #T_440a4_level1_row17 {
  background-color: rgba(246, 112, 136, 0.2);
}
#T_440a4_level1_row1, #T_440a4_level1_row18 {
  background-color: rgba(247, 117, 67, 0.2);
}
#T_440a4_level1_row2, #T_440a4_level1_row19 {
  background-color: rgba(213, 140, 49, 0.2);
}
#T_440a4_level1_row3, #T_440a4_level1_row20 {
  background-color: rgba(187, 151, 49, 0.2);
}
#T_440a4_level1_row4, #T_440a4_level1_row21 {
  background-color: rgba(163, 159, 49, 0.2);
}
#T_440a4_level1_row5, #T_440a4_level1_row22 {
  background-color: rgba(135, 167, 49, 0.2);
}
#T_440a4_level1_row6, #T_440a4_level1_row23 {
  background-color: rgba(79, 176, 49, 0.2);
}
#T_440a4_level1_row7, #T_440a4_level1_row24 {
  background-color: rgba(50, 176, 114, 0.2);
}
#T_440a4_level1_row8, #T_440a4_level1_row25 {
  background-color: rgba(52, 174, 144, 0.2);
}
#T_440a4_level1_row9, #T_440a4_level1_row26 {
  background-color: rgba(53, 172, 164, 0.2);
}
#T_440a4_level1_row10, #T_440a4_level1_row27 {
  background-color: rgba(54, 170, 181, 0.2);
}
#T_440a4_level1_row11, #T_440a4_level1_row28 {
  background-color: rgba(56, 168, 201, 0.2);
}
#T_440a4_level1_row12, #T_440a4_level1_row29 {
  background-color: rgba(59, 163, 236, 0.2);
}
#T_440a4_level1_row13, #T_440a4_level1_row30 {
  background-color: rgba(137, 148, 244, 0.2);
}
#T_440a4_level1_row14, #T_440a4_level1_row31 {
  background-color: rgba(186, 130, 244, 0.2);
}
#T_440a4_level1_row15, #T_440a4_level1_row32 {
  background-color: rgba(231, 102, 244, 0.2);
}
#T_440a4_level1_row16, #T_440a4_level1_row33 {
  background-color: rgba(245, 99, 211, 0.2);
}
#T_440a4_level0_row17 {
  background-color: rgba(253, 205, 172, 0.2);
}
</style>

  ----------- ---------------- --------------------
  Contigs     L10              41
              L20              99
              L30              174
              L40              267
              L50              385
              N10              12643769
              N20              9681846
              N30              7895799
              N40              6288966
              N50              4902968
              gc_content       35.248103813419206
              longest          43529772
              mean             1552486.9914285715
              median           331935.0
              sequence_count   4200
              shortest         1000
              total_bps        6520445364
  Scaffolds   L10              0
              L20              0
              L30              1
              L40              2
              L50              3
              N10              1438277616
              N20              1438277616
              N30              915491830
              N40              607508155
              N50              518932092
              gc_content       35.248103813419206
              longest          1438277616
              mean             3212576.533990148
              median           62362.5
              sequence_count   2030
              shortest         1000
              total_bps        6521530364
  ----------- ---------------- --------------------

### DNA Zoo’s Table, Reproduced

In [ ]:
library(reticulate)

Importing the library makes the code a little bit cleaner for inserting the values into the table below. For example, I would have had to type 4,902,968, but now I can just type 4,902,968 into the individual cells. I needed to convert the values into r objects, as the knitr engine used in rendering this document does not seem display output from execution of inline Python code directly.

|  |  |  |  |
|------------------|------------------|------------------|-------------------|
| **Contig length (bp)** | **Number of contigs** | **Contig N50 (bp)** | **Longest contig (bp)** |
| 6,520,445,364 | 4,200 | 4,902,968 | 43,529,772 |
| **Scaffold length (bp)** | **Number of scaffolds** | **Scaffold N50 (bp)** | **Longest scaffold (bp)** |
| 6,521,530,364 | 2,030 | 518,932,092 | 1,438,277,616 |

## :x NutFrame

In [ ]:
# A simple call on the Styler object
forma_df.data

                                       Value
Category  Label                             
Contigs   L10                             41
          L20                             99
          L30                            174
          L40                            267
          L50                            385
          N10                       12643769
          N20                        9681846
          N30                        7895799
          N40                        6288966
          N50                        4902968
          gc_content      35.248103813419206
          longest                   43529772
          mean            1552486.9914285715
          median                    331935.0
          sequence_count                4200
          shortest                      1000
          total_bps               6520445364
Scaffolds L10                              0
          L20                              0
          L30                              1
          